# CFN accuracy experiments (Colab)
Notes and runnable cells to iterate on the Causal Fusion Network (CFN), test detection accuracy, and try improvements safely.

## Embedding-aware CFN (TCN + Wav2Vec2)
- Ensure your CSV includes `tcn_visual_emb` and `wav2vec_audio_emb`.
- Train with `--use-embeddings` to produce `models/cfn_emb.pth`.
- Enable inference with: `CFN_USE_EMBEDDINGS=true` and `CFN_EMB_MODEL_PATH=models/cfn_emb.pth`.
- Optional SCM score: set `CFN_ENABLE_SCM_CHECKS=true`.


## 0) Runtime
- Runtime type: GPU (T4/A100 preferred)
- Python: 3.9+
- Enable persistent session if running sweeps

In [2]:
# 1) Environment setup
!git clone https://github.com/<your-org>/CausalX-Project.git
%cd CausalX-Project/backend
!pip install -r backend/requirements.txt

/bin/bash: line 1: your-org: No such file or directory
[Errno 2] No such file or directory: 'CausalX-Project/backend'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'backend/requirements.txt'


In [ ]:
# Optional: mount Drive if your datasets/models live there
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Copy model weights into backend/models
!cp /content/drive/MyDrive/path/to/cfn.pth models/cfn.pth

## 2) Smoke test on one video

In [ ]:
from src.cvi.cfn_frame_inference import run_cfn_on_video
from src.cvi.api.inference_service import summarize_video

video = "/content/drive/MyDrive/videos/sample.mp4"  # TODO: update

frames = run_cfn_on_video(
    video,
    threshold=0.6,
    chunk_seconds=10,
    max_seconds=None,  # set to cap processing
)

video_fake, fake_conf, highlight = summarize_video(
    frames,
    prob_thresh=0.6,
    ratio_thresh=0.3,
)
print(f"frames={len(frames)}, video_fake={video_fake}, conf={fake_conf:.3f}")

## 3) Batch evaluation helper

In [ ]:
import glob
from tqdm import tqdm
from src.cvi.api.inference_service import summarize_video
from src.cvi.cfn_frame_inference import run_cfn_on_video

def evaluate_dir(video_dir, prob_thresh=0.6, ratio_thresh=0.3):
    rows = []
    for path in tqdm(glob.glob(f"{video_dir}/*.mp4")):
        frames = run_cfn_on_video(path, threshold=prob_thresh, chunk_seconds=10)
        label, conf, _ = summarize_video(frames, prob_thresh=prob_thresh, ratio_thresh=ratio_thresh)
        rows.append({"video": path, "video_fake": label, "fake_conf": conf})
    return rows

rows = evaluate_dir("/content/drive/MyDrive/datasets/val")
rows[:3]

## 4) Techniques to test
Each section below has a short description and a runnable cell.

### 4.1 Threshold sweep (prob_thresh, ratio_thresh)
Grid-search operating points; compute simple metrics.

In [ ]:
import itertools
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

# Assumes rows with ground truth labels added: rows_gt = [{"video":..., "gt":0/1}]

prob_grid = [0.4, 0.5, 0.6, 0.7]
ratio_grid = [0.2, 0.3, 0.4]

def sweep_thresholds(rows_gt, prob_grid, ratio_grid):
    records = []
    for p, r in itertools.product(prob_grid, ratio_grid):
        preds = evaluate_dir("/content/drive/MyDrive/datasets/val", prob_thresh=p, ratio_thresh=r)
        merged = pd.DataFrame(rows_gt).merge(pd.DataFrame(preds), on="video")
        prec, rec, f1, _ = precision_recall_fscore_support(merged["gt"], merged["video_fake"], average="binary")
        records.append({"prob_thresh": p, "ratio_thresh": r, "precision": prec, "recall": rec, "f1": f1})
    return pd.DataFrame(records)

# rows_gt = [...]
# sweep_df = sweep_thresholds(rows_gt, prob_grid, ratio_grid)
# sweep_df.sort_values("f1", ascending=False).head()

### 4.2 Temporal smoothing
Apply a moving average over frame-level `fake_prob` to reduce spikes before summarizing.

In [ ]:
import numpy as np
from scipy.ndimage import uniform_filter1d
from src.cvi.api.inference_service import summarize_video

def summarize_with_smoothing(frames, prob_thresh=0.6, ratio_thresh=0.3, window=5):
    probs = np.array([f["fake_prob"] for f in frames])
    smooth = uniform_filter1d(probs, size=window, mode="nearest")
    for f, s in zip(frames, smooth):
        f["fake_prob_smooth"] = float(s)
    return summarize_video(frames, prob_thresh=prob_thresh, ratio_thresh=ratio_thresh)

# Example: video_fake, fake_conf, highlight = summarize_with_smoothing(frames, window=7)

### 4.3 Chunk sizing
Compare `chunk_seconds` (e.g., 5/10/15) for latency vs. stability.

In [ ]:
import time

def benchmark_chunks(video_path, chunk_sizes=(5, 10, 15)):
    results = []
    for cs in chunk_sizes:
        start = time.time()
        frames = run_cfn_on_video(video_path, threshold=0.6, chunk_seconds=cs)
        elapsed = time.time() - start
        results.append({"chunk_seconds": cs, "frames": len(frames), "seconds": elapsed})
    return results

# benchmark_chunks(video)

### 4.4 Audio/video lag search
Evaluate small AV shifts to reduce desync noise.

In [ ]:
import numpy as np

def av_lag_scores(frames, lags_ms=(-120, -60, 0, 60, 120)):
    scores = []
    base_lips = np.array([f["lip_aperture"] for f in frames])
    for lag in lags_ms:
        shifted_audio = np.array([f["audio_rms"] for f in frames])
        # crude shift by rolling; for precise shift re-run feature extraction with offset
        shift_frames = int(lag / 1000 * 30)  # assumes ~30 fps
        shifted_audio = np.roll(shifted_audio, shift_frames)
        corr = np.corrcoef(base_lips, shifted_audio)[0, 1]
        scores.append({"lag_ms": lag, "corr": float(corr)})
    return scores

# av_lag_scores(frames)

### 4.5 Feature weighting / model variants
Hook to rescale features before CFN to prototype weighting; useful if you add more cues.

In [ ]:
def run_with_feature_scale(video_path, av_scale=1.0, phys_scale=1.0):
    frames = run_cfn_on_video(video_path, threshold=0.6)
    for f in frames:
        f["av_mismatch"] *= av_scale
        f["jitter"] = f.get("jitter", 0.0) * phys_scale
    return frames

# frames_scaled = run_with_feature_scale(video, av_scale=1.2, phys_scale=0.8)

### 4.6 Augmentations for fine-tuning
Example: mild audio speed perturbation and landmark jitter to stress-test robustness (use for training, not inference).

In [ ]:
import librosa
import numpy as np

def augment_audio_speed(y, rate=1.02):
    return librosa.effects.time_stretch(y, rate=rate)

def jitter_landmarks(pts, sigma=0.002):
    noise = np.random.normal(scale=sigma, size=pts.shape)
    return pts + noise

# Use these in your data loader when re-training CFN

### 4.7 Bounding-box gating
Visual check: bboxes only where `fake_prob >= threshold`.

In [ ]:
import cv2

def draw_bboxes(video_path, frames, out_path="annotated.mp4"):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    fps = cap.get(cv2.CAP_PROP_FPS) or 25
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

    frame_map = {int(f["timestamp"] * fps): f for f in frames}

    idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        info = frame_map.get(idx)
        if info and info.get("bbox"):
            x1, y1, x2, y2 = info["bbox"]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        writer.write(frame)
        idx += 1

    cap.release()
    writer.release()

# draw_bboxes(video, frames, out_path="annotated.mp4")

## 5) Logging results
- Keep a CSV of experiment configs and metrics (thresholds, chunk size, smoothing window).
- Save short snippets of failure cases for qualitative review.